### MP2 Solution ###

In [1]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import os


In [6]:
# Load the dataset
audio_dir = '../MP2_Audio_Analytics/valve/' 

# Define a function to extract features ---
def extract_features(audio_file):
    """
    Extracts features from an audio file.

    Args:
      audio_file (str): Path to the audio file.

    Returns:
      list: A list of extracted features.
    """
    try:
        # Load the audio file
        y, sr = librosa.load(audio_file)

        # Extract features (you can add more)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
        mfccs_mean = np.mean(mfccs, axis=1)
        mfccs_std = np.std(mfccs, axis=1)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        chroma_mean = np.mean(chroma_stft, axis=1)
        chroma_std = np.std(chroma_stft, axis=1)
        spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
        zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y))
        rmse = np.mean(librosa.feature.rms(y=y))

        # Combine all features into a single list
        features = np.concatenate([
            mfccs_mean, mfccs_std, chroma_mean, chroma_std,
            [spectral_centroid, spectral_bandwidth, spectral_rolloff, 
             zero_crossing_rate, rmse]
        ])

        return features

    except Exception as e:
        print(f"Error processing {audio_file}: {e}")
        return None

In [7]:
# Extract features for all audio files 
features = []
labels = []

for id_folder in os.listdir(audio_dir):
    id_path = os.path.join(audio_dir, id_folder)
    if os.path.isdir(id_path): 
        for state_folder in ['normal', 'abnormal']:
            state_path = os.path.join(id_path, state_folder)
            for filename in os.listdir(state_path):
                if filename.endswith('.wav'):
                    audio_file = os.path.join(state_path, filename)
                    extracted_features = extract_features(audio_file)
                    if extracted_features is not None:
                        features.append(extracted_features)
                        labels.append(state_folder)

# Create a Pandas DataFrame 
df = pd.DataFrame(features)
df['label'] = labels

# Prepare the data for sklearn 
X = df.drop('label', axis=1)
y = df['label']

In [8]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [9]:
# Train a classifier 
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Make predictions and evaluate ---
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {100 * accuracy:.2f}%")

Accuracy: 93.76%


In [10]:
# Function for predicting on new data

def predict_anomaly(audio_file, clf, scaler):
    """
    Predicts whether an audio file contains an anomaly using the trained classifier.

    Args:
      audio_file (str): Path to the audio file.
      clf: The trained sklearn classifier.
      scaler: The fitted sklearn scaler.

    Returns:
      str: "normal" or "abnormal"
    """
    try:
        # Extract features from the new audio file
        features = extract_features(audio_file) 

        # Scale the features using the same scaler used for training
        features_scaled = scaler.transform(features.reshape(1, -1)) 

        # Make the prediction
        prediction = clf.predict(features_scaled)[0]

        return prediction

    except Exception as e:
        print(f"Error predicting {audio_file}: {e}")
        return None

In [12]:
# Prediction 1:
new_audio_file = '../MP2_Audio_Analytics/Valve_Data_for_Prediction/Valve1_000NB.wav'
prediction = predict_anomaly(new_audio_file, clf, scaler)
print(f"Prediction for {new_audio_file}: {prediction}")

Prediction for ../MP2_Audio_Analytics/Valve_Data_for_Prediction/Valve1_000NB.wav: normal


In [13]:
# Prediction 2:
new_audio_file = '../MP2_Audio_Analytics/Valve_Data_for_Prediction/Valve2_000AB.wav'
prediction = predict_anomaly(new_audio_file, clf, scaler)
print(f"Prediction for {new_audio_file}: {prediction}")

Prediction for ../MP2_Audio_Analytics/Valve_Data_for_Prediction/Valve2_000AB.wav: abnormal
